In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("abebooks.csv")
df.sample(3)

,isbn,title,seller,seller_country,price,author
8147,9781602645165,Side Effects: Death. Confessions of a Pharma-I...,Save With Sam,USA,166.21,"Virapen, John"
1413,9780691117362,Landscape with Rowers_ Poetry from the Netherl...,San Francisco Book Company,Frankreich,30.00,"Cotetzee, J. M. (trans & intro); et al (text)"
5806,9781107539303,Advanced Grammar in Use Book with Answers and ...,Byrd Books,USA,67.31,"Hewings, Martin"


In [3]:
df["isbn"].nunique()

1000

In [4]:
t = df.groupby("isbn", as_index=False).agg({"seller": "unique"})

In [5]:
t[t["seller"].apply(lambda x: "Bookbot" not in x)]["isbn"].values

array([], dtype=int64)

#### Jak často je Bookbot jediným prodávajícím?

In [9]:
sellers_by_isbn = df.groupby("isbn", as_index=False).agg(
    n_sellers=("seller", "nunique")
)
one_seller_offers = len(sellers_by_isbn[sellers_by_isbn["n_sellers"] == 1])

In [10]:
print(
    f"Bookbot je jediným prodávajícím v {(one_seller_offers/df['isbn'].nunique())*100}% případů"
)

Bookbot je jediným prodávajícím v 10.2% případů


#### Jak často je cena od Bookbota mezi 25% nejlevnějšími a nejdražšími nabídkami?

In [11]:
price_quantiles = (
    df.groupby("isbn")["price"].quantile([0.25, 0.75]).unstack().reset_index()
)
prices = df[df["seller"] == "Bookbot"].merge(price_quantiles, on="isbn", how="left")

prices["below_25p"] = prices["price"] < prices[0.25]
prices["above_75p"] = prices["price"] > prices[0.75]

In [12]:
prices["below_25p"].sum()

489

In [13]:
prices["above_75p"].sum()

152

#### Identifikujte prodejce, kteří mají častěji nižší cenu než BookBot.

In [14]:
bookbot_prices = df[df["seller"] == "Bookbot"][["isbn", "price"]]
competition_prices = df[df["seller"] != "Bookbot"][["seller", "isbn", "price"]]

In [15]:
price_vs_competition = competition_prices.merge(
    bookbot_prices, on="isbn", suffixes=("", "_bookbot")
)
price_vs_competition["cheaper_than_bookbot"] = (
    price_vs_competition["price"] < price_vs_competition["price_bookbot"]
)
price_vs_competition.groupby("seller", as_index=False).agg(
    cheaper_books=("cheaper_than_bookbot", "sum")
).nlargest(20, "cheaper_books")

,seller,cheaper_books
800,medimops,90
196,Book Deals,62
803,moluna,62
375,GreatBookPricesUK,59
346,"GF Books, Inc.",56
214,Books Unplugged,51
374,GreatBookPrices,50
252,Byrd Books,41
616,Revaluation Books,41
13,AHA-BUCH GmbH,40


#### Vypočítejte průměrnou odchylku nabídek Bookotu od nejnižší ceny.

In [16]:
min_price = df.groupby("isbn", as_index=False).agg({"price": "min"})
price_vs_min = df[df["seller"] == "Bookbot"].merge(
    min_price, on="isbn", suffixes=("", "_mininum")
)
price_vs_min["diff"] = price_vs_min["price"] - price_vs_min["price_mininum"]

In [17]:
price_vs_min["diff"].mean()

18.983849999999997

In [18]:
price_vs_min["diff"].median()

3.0

#### Analyzujte relativní zastoupení nabídek ze zahraničí ve stažených datech.

In [19]:
print(
    f'Podíl zahraničních nabídek v datech je {(len(df[df["seller_country"]!="Deutschland"])/len(df))*100}%'
)

Podíl zahraničních nabídek v datech je 84.408014571949%
